<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer:
Model features. Raw data for the network. 
### Hidden Layer:
Literally hidden layers within the model.
### Output Layer:
Target variables. This is what is predicted. There are always the same number of output nodes as there are possible output classifications.
### Neuron:
They take in inputs multiplies them by a weight, then performs an activation function and sends the output to the next layer of the network.
### Weight:
The coefficient of a neuron - typically derived from the data.
### Activation Function:
This function determines the amount of signal to pass to the next layer from any given neuron. Sigmoid, for example.
### Node Map:
Visual representation of the structure of a neural net.
### Perceptron:
Very simple neural net. Typically a just an input layer and an output layer.

## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

Using the home price example:

There are 3 inputs when determining price -- sq ft, number of bathrooms, number of bedrooms:

input1 = 2000sq ft, 
weight1 = 180 (average price per sq ft of data) 

input2 = 4 bathrooms, 
weight2 = 200 (average price per bathroom of data) 

input3 = 6 bedrooms, 
weight3 = 350 (average price per bedroom of data) activation function = sigmoid bias = 175 (picked arbitrarily)

output = sigmoid((weight1 input1) + (weight2 input2) + (weight3 * input3) + bias)

![alt text](https://schwalbe10.github.io/thinkage/images/posts/perceptron.png)

## Write your own perceptron code that can correctly classify (99.0% accuracy) a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


data = { 'x1': [0,1,0,1],
         'x2': [0,0,1,1],
         'y':  [1,1,1,0]
       }

df = pd.DataFrame.from_dict(data).astype('int')

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1-sx)

In [3]:
inputs = np.array(df[['x1', 'x2']])
outputs = np.array(df[['y']])
print(inputs)
outputs

[[0 0]
 [1 0]
 [0 1]
 [1 1]]


array([[1],
       [1],
       [1],
       [0]])

In [4]:
bias = 2 * np.random.random((1,1)) - 1
weights = 2 * np.random.random((2,1)) - 1
weighted_sum = np.dot(inputs, weights) + bias

activated_output = sigmoid(weighted_sum)

error = outputs - activated_output

adjusted = error * sigmoid_derivative(activated_output)

In [5]:
for iteration in range(10000):
    # Weighted sum of inputs / weights
    weighted_sum = np.dot(inputs, weights) + bias
    # Activate!
    activated_output = sigmoid(weighted_sum)
    # Cac error
    error = outputs - activated_output
    adjustments = error * sigmoid_derivative(activated_output)
    # Update the bias
    bias += np.mean(error)
    # Update the Weights
    weights += np.dot(inputs.T, adjustments)
    

print("Weights after training")
print(weights)

print("Output after training")
print(activated_output)

Weights after training
[[-12.5528237]
 [-12.5528237]]
Output after training
[[0.99999999]
 [0.99833612]
 [0.99833612]
 [0.0021169 ]]


## Implement your own Perceptron Class and use it to classify a binary dataset: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [6]:
diabetes = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Although neural networks can handle non-normalized data, scaling or normalizing your data will improve your neural network's learning speed. Try to apply the sklearn `MinMaxScaler` or `Normalizer` to your diabetes dataset. 

In [7]:
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.metrics import accuracy_score


feats = list(diabetes)[:-1]

X = np.array(diabetes[feats])
y = np.array(diabetes[['Outcome']])

MinMax = MinMaxScaler()
X = MinMax.fit_transform(X)

In [8]:
class Perceptron:
    
    def __init__(self, niter = 10):
        self.niter = niter
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        sx = self.sigmoid(x)
        return sx * (1-sx)

    def fit(self, X, y):
        """Fit training data
        X : Training vectors, X.shape : [#samples, #features]
        y : Target values, y.shape : [#samples]
        """

        # Randomly Initialize Weights
        self.weights = 2 * np.random.random((X.shape[1], 1)) -1
    
        # Randomly Initialize Bias
        self.bias = 2 * np.random.random() - 1

        for i in range(self.niter):
        
            # Weighted sum of inputs / weights
            weighted_sum = np.dot(X, self.weights) + self.bias
        
            # Activate!
            activated_output = self.sigmoid(weighted_sum)
        
            # Calc error
            error = y - activated_output
            
            # Update the Weights & Bias
            adjustments = error * self.sigmoid_derivative(activated_output)
            self.weights += np.dot(X.T, adjustments)
            self.bias += np.mean(error)

    def predict(self, X):
        """Return class label after unit step"""
        
        weighted_sum = np.dot(X, self.weights) + self.bias
        return np.round(self.sigmoid(weighted_sum))

In [9]:
model = Perceptron(niter=10)
model.fit(X, y)
test = model.predict(X).astype(int)

In [10]:
print(f"Accuracy Score: {100 * accuracy_score(test, y) :.2f}")

Accuracy Score: 49.87


## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?

In [12]:
# I wrote this library
from Fortuna import RelativeWeightedChoice

In [13]:
class ActiveChoice:

    def __init__(self, name, data, start=50, lo=1, hi=100):
        self.name = name
        self.raw_data = {k: start for k in data}
        self.start = start
        self.lo = lo
        self.hi = hi
        self.data = self.build()
        self.wins = 0.0
        self.losses = 0.0

    def __call__(self):
        return self.data()

    def build(self):
        return RelativeWeightedChoice(
            zip(self.raw_data.values(), self.raw_data.keys()))

    def winner(self, result):
        self.wins += 1
        if self.raw_data[result] < self.hi:
            self.raw_data[result] += 1
            self.data = self.build()

    def looser(self, result):
        self.losses += 1
        if self.raw_data[result] > self.lo:
            self.raw_data[result] -= 1
            self.data = self.build()

    def win_ratio(self):
        total_games = self.wins + self.losses
        return self.wins / total_games if total_games else total_games

    def __str__(self):
        max_weight = max(self.raw_data.values())
        output = (
            f"{self.name}: {self.raw_data}",
            f"Best strategies: " + ", ".join(
                k for k, v in self.raw_data.items() if v == max_weight),
            f"Win Rate: {self.win_ratio():.2%}",
            ""
        ) if self.win_ratio() else (f"{self.name}: {self.raw_data}", "")
        return "\n".join(output)

In [14]:
def run_game(team_a, team_b, cycles=10000):
    game_rules = {
        "rock": {
            "rock": 0, "paper": -1, "scissors": 1, "lizard": 1, "spock": -1},
        "paper": {
            "rock": 1, "paper": 0, "scissors": -1, "lizard": -1, "spock": 1},
        "scissors": {
            "rock": -1, "paper": 1, "scissors": 0, "lizard": 1, "spock": -1},
        "lizard": {
            "rock": -1, "paper": 1, "scissors": -1, "lizard": 0, "spock": 1},
        "spock": {
            "rock": 1, "paper": -1, "scissors": 1, "lizard": -1, "spock": 0},
    }

    def game():
        a = team_a()
        b = team_b()
        this_game = game_rules[a][b]
        if this_game > 0:
            team_a.winner(a)
            team_b.looser(b)
        elif this_game < 0:
            team_a.looser(a)
            team_b.winner(b)
        else:
            team_a.looser(a)
            team_b.looser(b)

    for _ in range(cycles):
        game()

In [15]:
team_1 = ActiveChoice("Team 1", ("lizard", "scissors"))
team_2 = ActiveChoice("Team 2", ("rock", "paper", "spock"))

print("# Pre Game\n")
print(team_1)
print(team_2)
run_game(team_1, team_2)

print("# Final Scores\n")
print(team_1)
print(team_2)

# Pre Game

Team 1: {'lizard': 50, 'scissors': 50}

Team 2: {'rock': 50, 'paper': 50, 'spock': 50}

# Final Scores

Team 1: {'lizard': 1, 'scissors': 1}
Best strategies: lizard, scissors
Win Rate: 9.03%

Team 2: {'rock': 100, 'paper': 1, 'spock': 3}
Best strategies: rock
Win Rate: 90.97%

